In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.autonotebook import tqdm
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split

<ipython-input-1-38939910a397>:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
url = 'https://raw.githubusercontent.com/DavGev/OMDS_project/master/data.txt'
data = pd.read_csv(url)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X = data.loc[data.Y.isin(['A', 'G']), data.columns != 'Y']
y = data.loc[data.Y.isin(['A', 'G']), 'Y']
y = pd.get_dummies(y)['G']

X = X.to_numpy()
y = y.to_numpy()


In [ ]:
class Sigmoid:
    def __call__(self, x):
        return 1 / (1 + np.exp(-x)) # dtype=np.float128 to prevent rounding up to 0 or 1

    def grad(self, s_x):
        return s_x * (1 - s_x)


class Tanh:
    def __init__(self, sigma):
        self.sigma = sigma

    def __call__(self, x):
        return np.tanh(self.sigma * x)

    def grad(self, th_x):
        return self.sigma * (1 - th_x ** 2)


class CrossEntropy:
    def __call__(self, y, p):
        return - (y * np.log(p) + (1-y) * np.log(1-p)).mean()

    def grad(self, y, p):
        return - (y / p - (1-y) / (1-p)) / y.shape[0]

cross_entropy = CrossEntropy()


# PART 1 - MLP

- $ E(ω;π) = - \frac{1}{P} ∑_{i=1}^P {[y_i \ln(p_i) + (1 - y_i) \ln(1 - p_i)]} + ρ \|ω\|^2 $
- $ ρ = 10^{−4} $
- $ S(v)_j = \frac{e^{v_j}}{∑_{h=1}^n e^{v_h}} $
- The activation function $g(t) := tanh(t)$

### Hyperparameters
- the number H of hidden layers (max. 4) (only for question 1)
- the number of neurons N of the hidden layers
- the spread $σ > 0$ in the activation function $g$ ($g$ is available in Python with $σ = 1$: `numpy.tanh`)

### Tasks
- Write a program which implements the regularized training error function $E(v,w,b)$
- **Question 1. (grade up to 20)** Use an optimization algorithm from `scipy.optimize` that uses the gradient to determine the parameters $v_j ,w_{ji}, b_j$ which minimize the error.
- **Question 2. (grade up to 10)** Develop an RBF neural network trained by implementing the decomposition method studied in class.

| Ex | H | N | $σ$ | $ρ$ | Optimization | Message | Init train error | Final train error | Final  test error | f\grad evaluations | Time |
| -|-|-|-|-|-|-|-|-|-|-|-|
| Q1 Full MLP |
| Q2 RBF |


\* optimization: with parameters (optimality accuracy, max number of iterations etc)

\* message: in output (successful optimization or others, number of iterations, number of function/gradient evaluations, starting/final value of the objective function, starting/final accuracy etc)

In [ ]:
class MLPLayer():
    def __init__(self, input_size, output_size, activation):
        self.w = np.random.random((output_size, input_size))
        self.w /= (self.w ** 2).sum() ** 0.5
        self.activation = activation

        self.input = None
        self.output = None
        self.grad_w = None
        self.grad_input = None


    def Forward(self, input):
        self.input = np.insert(input, 0, 1, axis=-1)
        sum = self.input @ self.w.T
        self.output = self.activation(sum)


    def Backward(self, grad_output):
        grad_sum = self.activation.grad(self.output) * grad_output
        self.grad_w = grad_sum.T @ self.input
        self.grad_input = grad_sum @ self.w[:,1:]


In [ ]:
class MLP():
    def __init__(self, N, sigma=10):
        '''
        N: array of numbers of neurons in the input layer,
        each hiden layer and the output layer

        For example if our data is 10 dimentional, we need two hidden layers
        with 5 neurons, and we have 2 classes, than N = [10, 5, 5, 2]
        '''
        self.rho = 1e-4
        self.layers = [
            MLPLayer(
                input_size = N[i] + 1,
                output_size = N[i+1],
                activation = Tanh(sigma)
            ) for i in range(len(N) - 1)
        ]
        self.layers[-1].activation = Sigmoid()


    def assign_w(self, w):
        '''
        w: flattened array of all the weights
        '''
        start = 0
        end = 0
        for layer in self.layers:
            end += layer.w.size
            layer.w = w[start : end].reshape(layer.w.shape)
            start = end


    def get_flat(self, what):
        if what == 'w':
            return np.concatenate([layer.w.flatten() for layer in self.layers])
        if what == 'grad_w':
            return np.concatenate([layer.grad_w.flatten() for layer in self.layers])


    def predict(self, X):
        ipnut = X
        for layer in self.layers:
            layer.Forward(ipnut)
            ipnut = layer.output
        return ipnut


    def error(self, X, y):
        y = y.reshape([-1, 1])
        p = self.predict(X)
        error = cross_entropy(y, p)
        error += self.rho * (self.get_flat('w') ** 2).sum()
        return error


    def gradient(self, X, y):
        y = y.reshape([-1, 1])
        p = self.predict(X)
        grad_output = cross_entropy.grad(y, p)
        for layer in self.layers[::-1]:
            layer.Backward(grad_output)
            grad_output = layer.grad_input

        grad = self.get_flat('grad_w')
        grad += 2 * self.rho * self.get_flat('w')
        return grad


    def fit(self, X, y, method='trust-constr'):

        def fun(w):
            self.assign_w(w)
            return self.error(X, y)

        def jac(w):
            self.assign_w(w)
            return self.gradient(X, y)

        w0 = self.get_flat('w')
        message = minimize(fun=fun, jac=jac, x0=w0, method=method)
        self.assign_w(message.x)
        return message


    def accuracy(self, X, y, threshold=0.5):
        y = y.reshape([-1, 1])
        p = self.predict(X)
        return ((p > threshold) == y).mean()


### Some observations on the minimization methods

|method|speed|comment|
|-|-|-|
|`'Nelder-Mead'`|slow|stabile, accuracy = 0.505|
|`'Powell'`|very slow| accuracy > 0.93|
|`'CG'`|very slow|very unstabile, sometimes accuracy < 0.5|
|`'BFGS'`|fast| unstabile, sometimes accuracy = 0.505|
|`'Newton-CG'`|not too fast| quite stabile, mostly accuracy > 0.97|
|`'L-BFGS-B'`||just an error|
|`'TNC'`|very fast| quite stabile, mostly accuracy > 0.96|
|`'COBYLA'`|too fast| mostly accuracy = 0.505|
|`'SLSQP'`|lightning fast| accuracy = 0.505|
|`'trust-constr'`|not too fast| mostly accuracy > 0.995|
|`'dogleg'`||we need Hessian|
|`'trust-ncg'`||we need Hessian|
|`'trust-exact'`||we need Hessian|
|`'trust-krylov'`||we need Hessian|

After consideration, I chose `'trust-constr'` to be the default method of minimization

## Cross Validation

In [ ]:
np.random.seed(42)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
np.seterr(divide='ignore', invalid='ignore')

models_data = []
for N_layers in tqdm(range(5)):
    for N_neurons in tqdm(range(4, 17, 4) if N_layers != 0 else range(1)):
        N = [16] + [N_neurons] * N_layers + [1]

        for log_sigma in tqdm(range(-2, 3)):
            sigma = 10 ** log_sigma

            kf = KFold(n_splits=5, shuffle=True)
            kf.get_n_splits(X_train_val)

            for train_index, valid_index in kf.split(X_train_val):
                scaler = StandardScaler()
                model = MLP(N, sigma)

                X_train = X_train_val[train_index]
                y_train = y_train_val[train_index]
                X_val = X_train_val[valid_index]
                y_val = y_train_val[valid_index]

                X_train_scaled = scaler.fit_transform(X_train)
                model.fit(X_train_scaled, y_train)
                train_error = model.error(X_train_scaled, y_train)

                X_val_scaled = scaler.transform(X_val)
                val_error = model.error(X_val_scaled, y_val)

                models_data.append({'N_layers': N_layers,
                                    'N_neurons': N_neurons,
                                    'sigma': sigma,
                                    'train_error': train_error,
                                    'val_error': val_error})

                pd.DataFrame(models_data).to_csv('/content/drive/MyDrive/Colab Notebooks/models_data.csv')

# np.seterr(divide='warn', invalid='warn')

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
models_data = pd.DataFrame(models_data)
models_data

,N_layers,N_neurons,sigma,train_error,val_error
0,0,0,0.01,0.029154,0.037239
1,0,0,0.01,0.027557,0.045298
2,0,0,0.01,0.031753,0.020846
3,0,0,0.01,0.019813,0.090461
4,0,0,0.01,0.029797,0.032322
...,...,...,...,...,...
420,4,16,100.00,0.051360,0.033528
421,4,16,100.00,0.000630,0.000630
422,4,16,100.00,0.201583,0.179627
423,4,16,100.00,0.097632,0.146392


In [ ]:
grouped = models_data.groupby(['N_layers', 'N_neurons', 'sigma']).agg('mean').reset_index()

In [ ]:
idx = grouped.val_error.idxmin()
grouped.loc[idx]

N_layers       2.000000
N_neurons      8.000000
sigma          1.000000
train_error    0.003304
val_error      0.007887
Name: 32, dtype: float64

# Radial Basis Function (RBF) Network

**Question 2. (grade up to 10)** Develop an RBF neural network trained by implementing the decomposition method studied in class.

In [ ]:
class Gaussian():
    def __init__(self, sigma):
        self.sigma = sigma

    def __call__(self, x):
        return np.exp(- x ** 2 / self.sigma ** 2)

    def derivative(self, x):
        return - 2 * x / self.sigma ** 2 * self.__call__(x)


class Multiquadric():
    def __init__(self, sigma):
        self.sigma = sigma

    def __call__(self, x):
        return (x ** 2 + self.sigma ** 2) ** 0.5

    def derivative(self, x):
        return x / self.__call__(x)


class InverseMultiquadric():
    def __init__(self, sigma):
        self.sigma = sigma

    def __call__(self, x):
        return (x ** 2 + self.sigma ** 2) ** -0.5

    def derivative(self, x):
        return - x * self.__call__(x) ** 3


In [ ]:
class RBF_network():
    def __init__(self, input_size, N_centers, RBF, sigma=1, rho=1e-4):
        self.RBF = RBF(sigma)
        self.w = np.random.random(N_centers)
        self.w /= (self.w ** 2).sum() ** 0.5
        self.c = np.random.random([N_centers, input_size])
        self.c *= sigma / (self.c ** 2).sum() ** 0.5
        self.sigmoid = Sigmoid()
        self.rho = rho

        self.dist = None
        self.norm = None
        self.Phi = None


    def predict(self, X):
        self.dist = self.c[None, :, :] - X[:, None, :]
        self.norm = (self.dist ** 2).sum(axis=-1) ** 0.5
        self.Phi = self.RBF(self.norm)
        return self.sigmoid(self.Phi @ self.w)


    def error(self, X, y):
        p = self.predict(X)
        error = cross_entropy(y, p)
        error += self.rho * (self.w ** 2).sum()
        return error


    def grad_w(self, X, y):
        p = self.predict(X)
        grad = cross_entropy.grad(y, p)
        grad *= self.sigmoid.grad(p)
        grad = grad @ self.Phi
        return grad


    def grad_c(self, X, y):
        p = self.predict(X)
        grad = cross_entropy.grad(y, p)[:, None] * self.w[None, :]
        grad *= self.RBF.derivative(self.norm)
        grad = grad[:, :, None] * self.dist / self.norm[:, :, None]
        return grad.sum(axis=0)


    def fit(self, X, y, method='trust-constr'):
        while (self.grad_w(X, y)**2).sum() + (self.grad_c(X, y)**2).sum() > 1e-3:
            def fun_w(w):
                self.w = w
                return self.error(X, y)
            def jac_w(w):
                self.w = w
                return self.grad_w(X, y)

            message = minimize(fun=fun_w, jac=jac_w, x0=self.w, method=method)
            self.w = message.x

            def fun_c(c):
                self.c = c.reshape(self.c.shape)
                return self.error(X, y)
            def jac_c(c):
                self.c = c.reshape(self.c.shape)
                return self.grad_c(X, y).flatten()

            message = minimize(fun=fun_c, jac=jac_c, x0=self.c.flatten(), method=method)
            self.c = message.x.reshape(self.c.shape)


In [ ]:
model = RBF_network(input_size=16, N_centers=4, RBF=Gaussian, sigma=12)

In [ ]:
model.fit(X, y)